# 库存管理

## ODOO调用xml-rpc


In [ ]:
url = "http://39.105.154.26:8069"
db = "wms"
username = 'api@dellege.com'
password = "api"

连接，普通操作，无需登录

In [ ]:
import xmlrpc.client
common = xmlrpc.client.ServerProxy('{}/xmlrpc/2/common'.format(url))

登录服务器，获取user id

In [ ]:
uid = common.authenticate(db, username, password, {})

In [ ]:
models = xmlrpc.client.ServerProxy('{}/xmlrpc/2/object'.format(url))

## 库位设置
<!-- 仓库库别分类项次
库位码
类别成品A库成品 B库成品C库原材料库
库别码 -->
刘总：
按A1、B、…26个字母来标识每一排货架；与数字结合来按标识每一个库位；如:U12,表示第 U 排货架,第12格。

* 成品A：标签成品、外购PE袋、气泡袋等
* 成品B：说明书
* 成品C：设备类 
* 原材料：标签材料,铜版纸、PE 等

修改：
* 原有U12改为U-01-2。条码加前缀LOC-
* U第U通道（通道号，横排），系统标记x
* 01代表货架号（第01列，货架以列为单位），始终用二位数表示。系统标记y
* 2代表第2层。系统标记z

AGV：
* 入库码头，条码 LOC-AGV-in  后面加序号
* 出库码头，条码 LOC-AGV-out  

## 创建库位

In [ ]:
STOCK_LOCATION_TABLE='stock.location'

def create_stock_location(name,barcode,usage=None,x=0,y=0,z=0):
    count = models.execute_kw(db, uid, password,
        STOCK_LOCATION_TABLE, 'search_count',
        [[['name', '=', name]]])
    if count == 0:
        value = {'name': name,
                 'posx': x,
                 'posy': y,
                 'posz': z,
                 'barcode': barcode,
                }
        if usage is not None:
            value['usage'] = usage
        id = models.execute_kw(db, uid, password, 
                           STOCK_LOCATION_TABLE, 'create', 
                           [value])
        return id
    else:
        return -1
    

程序所有货架的xyz维度是一致的。这往往不是实际情况。

根据现场情况，进odoo界面删掉一些不存在的库位。

初始化库位：

In [ ]:
def init_stock_locations(xlen=2,ylen=2,zlen=2):
    for x in range(xlen):
        for y in range(ylen):
            for z in range(zlen):
                name = '{}-{:02d}-{}'.format(chr(x+ord('A')),y+1,z+1)                
                create_stock_location(name,'LOC-'+name, x+1,y+1,z+1)

init_stock_locations(1,1,2)

create_stock_location('AGV入库码头','LOC-AGV-in','transit')
create_stock_location('AGV出库码头','LOC-AGV-out','transit')

## 列出所有库位
此API供前端显示所有库位。

这里加一个限制，xyz都必须从下标1开始，中间编号不能中断。
如果xyz为0，认为不是我们管理的（一些虚拟库位）。

In [ ]:
def valid_locations_rules():
    return [['usage', '=', 'internal'],
        ['posx', '>', 0], ['posy', '>', 0], ['posz', '>', 0],
        ['barcode', '!=', False]]

def list_stock_locations():
    where = valid_locations_rules()
    select = {'fields': ['location_id', 'name', 'posx', 'posy', 'posz', 'barcode']}
    return models.execute_kw(db, uid, password,
                        STOCK_LOCATION_TABLE, 'search_read',
                        [where], select)

一般来说，一个通道以内的行列数应该是规整的；
但是不同通道因为场地空间限制，有可能各通道不同。
所以，返回值组织为列表，列表的元素是通道。
通道为二维数组，数组的行列代表货架号和层号。
数组的值为location_id，0代表库位无效。

方法是判断一个通道内，yz的最小最大值，直接生成数组。

In [ ]:
import numpy
def get_locations_array():
    inner = []
    locations = list_stock_locations()
    for l in locations:
        x = l['posx'] - 1
        y = l['posy'] - 1
        z = l['posz'] - 1
        if x > 100 or y > 30 or z > 16:
            return 'unsupported range'
        
        for i in range(len(inner), x+1):
            inner.append({'ymax': y, 
                          'zmax': z })
            
        if y > inner[x]['ymax']:
            inner[x]['ymax'] = y
        if z > inner[x]['zmax']:
            inner[x]['zmax'] = z
            
    for i in inner:
        i['locations'] = numpy.zeros((i['ymax']+1, i['zmax']+1), numpy.int)
        
    for l in locations:
        x = l['posx'] - 1
        y = l['posy'] - 1
        z = l['posz'] - 1
        inner[x]['locations'][y][z] = l['location_id']
    return inner



In [ ]:
get_locations_array()

In [ ]:
# 异步进程在jupyter正常工作。单独启动服务器不需要
import nest_asyncio
nest_asyncio.apply()

In [ ]:
from fastapi import FastAPI, Depends
app = FastAPI()

In [ ]:
@app.get("/stock_locations/")
def get_stock_locations_array():
    return get_locations_array()

## 库位是否为空
查询某些库位是否为空。这个状态供前端显示之用。

返回count记录数，每条记录代表该位置产品数量大于0。因此可以指代库位是否有产品。

In [ ]:
STOCK_QUANTITY_TABLE = 'stock.quant'

def compute_locations_is_empty(location_ids = []):
    if len(location_ids) == 0:
        for l in list_stock_locations():
            location_ids.append(l['id'])

    if len(location_ids) <= 0:
        return
    
    where = [
        ['location_id','in', location_ids],   # valid locations
        ['qty','>',0]   # not empty
    ]
    select = ['location_id_count','location_id']
    groupby = ['location_id']  
    return models.execute_kw(db, uid, password,
                        STOCK_QUANTITY_TABLE, 'read_group',
                        [where, select, groupby], [])

In [ ]:
compute_locations_is_empty()

# 包裹管理
以周转箱为基本单位。

德立基目前创建800个周转箱。

ODOO分为包裹类型和实例。
类型就一种：周转箱，所以id是常数1. 类型的缩写BOX。
实例自增编码，取五位数。

In [ ]:
PACKAGE_TABLE='stock.quant.package'

def create_package(name, typeid=1):
    where = [['name', '=', name]]
    count = models.execute_kw(db, uid, password,
        PACKAGE_TABLE, 'search_count',
        [where])
    
    if count == 0:
        id = models.execute_kw(db, uid, password, 
                           PACKAGE_TABLE, 'create', 
                           [{'name': name,
                             'packaging_id': typeid,
                             'barcode': False,
                            }])       
        return id
    else:
        return -1
    

In [ ]:
def init_packages(count):
    for i in range(count):
        name = 'BOX{:05d}'.format(i+1) 
        create_package(name)
        
init_packages(8)

## 包裹内产品修改

# 出入库

入库是指把周转箱用AGV搬入库位。
入库过程：给嘉腾系统的数据库写入参数：箱子编号、库位编号；
AGV自动来到码头，把箱子送到指定的位置。

对于WMS系统，变化是：入库码头→AGV→库位

在UI上显示入库和出库区。被搬运的周转箱用特殊颜色表示。

## 入库队列
入库的队列应放在WMS，我们建立一个虚拟的区域：入库码头。
入库码头有传送带，传送带按照顺序把周转箱上到AGV。
所以入库区有多个周转箱，根据传送带上面来考虑入库顺序，可以按照某个时间字段来orderby。

AGV虚拟库位，为了数据库查询，所有库位名称都必须以AGV起头，odoo命名必须按照以下严格同名，否则数据库不能工作。
* **AGV入库码头**：WMS设置
* **AGV出库码头**：与入库同理
* AGV：目前只有一个AGV，所以就一个编号

实现查询出入库码头的接口：返回in out队列的周转箱列表，按顺序排列。
1. 查询AGV出入库码头的location_id
1. 分组查询STOCK_QUANTITY_TABLE表，该location_id下有哪些package记录
1. 查询
打包BOX来分组

In [ ]:
# models.execute_kw(
#     db, uid, password, STOCK_LOCATION_TABLE, 'fields_get',
#     [], {'attributes': ['string', 'help', 'type']})

In [ ]:
def get_agv_locations():
    where = [['usage','=','transit'],['barcode','like','LOC-AGV-%']]
    select = {'fields': ['location_id', 'name', 'barcode']}
    return models.execute_kw(db, uid, password,
                        STOCK_LOCATION_TABLE, 'search_read',
                        [where], select)
    
def get_location_boxes(location_ids):
    if isinstance(location_ids, int): 
        location_ids = [location_ids]
#     location_ids = [68] #[a['id'] for a in agv]
#     print(ids)    
#     where = [['location_id','in', location_ids]]
#     select = ['package_id_count', 'package_id']
#     groupby = ['package_id']
#     packs = models.execute_kw(db, uid, password,
#                         STOCK_QUANTITY_TABLE, 'read_group',
#                         [where, select, groupby], [])
#     pack_ids = [p['package_id'][0] for p in packs]
#     print(pack_ids)
#     print(location_ids)
    select = {'fields': ['location_id', 'name', 'package_id', 'in_date', 'qty'],
              'order': 'in_date' }
    where = [
        ['location_id','in', location_ids], 
        ['package_id', '>=', 1],  # in the box
        ['qty','>',0]   # not empty
    ]    
    return models.execute_kw(db, uid, password,
                        STOCK_QUANTITY_TABLE, 'search_read',
                        [where], select)
#     print(products)
#     pid = [p['package_id'][0] for p in products]
#     print(pid)
    
agv=get_agv_locations()
print(agv)
ids = [a['id'] for a in agv]
get_location_boxes(67)

## 调拨
WMS称为**调拨**，其实质是记录货物的每一次移动。
* 找到一个空位置，源位置（AGV码头）到该空位增加一条预计调拨的记录。
* AGV搬运到位后，该调拨设置为完成状态。
* AGV如果搬运失败，调拨设置为取消状态。
* 如果空位存在预调拨，我们也认为该位置不为空。

### 空位查询
根据调拨规则，空位是：没有产品，且没有预计调拨。

目前选择空位的策略是随机选择空位，以便所有空位可以均衡使用。
未来需要增加其他策略：库存区规则，例如PDA只能放在A-01货架。

In [ ]:
def get_random_empty_location():
    where = [
        ['qty','>',0]   # not empty
    ]
    select = ['location_id']
    groupby = ['location_id']  
    not_empty = models.execute_kw(db, uid, password,
                        STOCK_QUANTITY_TABLE, 'read_group',
                        [where, select, groupby], [])
    print(not_empty)
    ids = [e['location_id'][0] for e in not_empty]
    
    print(ids)
    
#     not_empty = compute_locations_is_empty()
#     ids = [e['location_id'][0] for e in not_empty]
    
#     print(ids)

    select = {'fields': ['id', 'name'],
             'limit': 20}
    where = valid_locations_rules() 
    where.append(['id','not in', ids])
#     print(where)
    empty = models.execute_kw(db, uid, password,
                        STOCK_LOCATION_TABLE, 'search_read',
                        [where], select)
    print(empty)
    if len(empty) == 0:
        return None
    
    from random import randrange
    id = randrange(len(empty))
    return empty[id]
    
get_random_empty_location()

### 增加调拨

### 入库完成

In [ ]:
ids = models.execute_kw(db, uid, password,
    'product.template', 'search',
    [[]],
    {'limit': 30})

In [ ]:
[record] = models.execute_kw(db, uid, password,
    'product.template', 'read', [ids])
record